# Temporal Structure of a Time Series 

Typically the goal of a time series analysis is to implement some sort of forecasting model. To choose the right path forward, it is important to determine the **structure** of the time series you are working with! This includes:

- determine if your time series is **white noise** or a **random walk** to determine preictability 
- determine if your time series is **stationary** 
- determine if your time series has **seasonality** 

After determining these qualities, we can move on to removing noise where possible to make the underlying signal more obvious, and removing any seasonality if applicable. 

## Set up workspace 

Let's begin by setting up our workspace & importing some packages we can use to help us

In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
from pandas.tseries.frequencies import to_offset
import statsmodels as sm
import statsmodels.graphics.tsaplots as tsaplots
import statsmodels.tsa.stattools as stattools


## White noise 

Let's start by exploring a **white noise** time series. A white noise time series by definition fits these features: 

- a mean of zero 
- a mean that does not change over time 
- a variance/standard deviation that does not change over time 
- no autocorrelation at any/all time lags 

If your time series does **not** fit all these specifications it is likely it is not white noise and there is probably some underlying signal that can be extracted and it is possible to produce some sort of forecast model based on it. However, if the series is in fact white noise then we can't do any forecasting with it! 

Let's bring in a time series from `whitenoise.csv` and demonstrate some tests that will confirm if it is white noise or not. We will start with a look into the summary statistics, produce a histogram to display the distribution of values, & create a simple time series plot to get acquainted with this data. 

In [ ]:
whitenoise = pd.read_csv('whitenoise.csv')
whitenoise.hist()
whitenoise.describe()

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(whitenoise, c = 'k')
plt.grid(which = 'major')

We can see a few things such as the fact that the data has a near-zero mean of 0.03. From the histogram it also appears to be normally distributed, aka a Gaussian distribution. 

What about the time series plot? Do you notice any patterns or obvious signals? There seem to be various spikes and dips in data value which center on a value of zero & don't seem to follow any particular pattern. It looks completely random, but off course, an eyeball analysis is not enough! We need to prove this through statistical methods. 

Let's see how the mean changes over time. Let's calculate a moving average over every 50 values and plot it alongside the original time series. 

In [ ]:
whitenoise_roll = whitenoise.rolling(100).mean()
whitenoise_roll['offset'] = whitenoise_roll.index - 50

plt.figure(figsize=(12, 4))
plt.plot(whitenoise, c = 'k', label = 'Original')
plt.plot(whitenoise_roll.offset, whitenoise_roll.data, c = 'r', linewidth = 3, label = 'Rolling Average')
plt.grid(which = 'major')
plt.legend()
plt.title('White Noise')

It looks like the mean value is more or less the same through the entire time series! 

## Autocorrelation plot 

A very powerful tool we can utilize is an **autocorrelation plot**, sometimes referred to as a corellogram. This is used to determine the **randomness** of a dataset by comparing the data values against themselves (computing autocorrelations) at various time lag separations. Random data will have near-zero autocorrelations for any/all lag separations! This is useful in many contexts, but we will start by using it here in the context of checking if our series is white noise. 

We can easily produce an autocorrelation plot using the `tsaplots` package from `statsmodels` `graphics` package. Previously we imported this as `tsaplots` so now from that package we can call `plot_acf`. This plotting function requires two input arguments, the time series data to be autocorrelated as well as the number of lags. There is no fixed rule for the number of lags to use, and this will depend on your dataset. Here we will use 50. 

In [ ]:
tsaplots.plot_acf(whitenoise, lags=50);

Our ACF plot shows entirely near-zero values which **confirms that this time series is white noise!** There is no correlation between the values in the dataset and those that came before/after. It is entirely random. 

## Random walk 

Now that we know how to identify white noise, let's learn how to identify when a time series is a **random walk**. A random walk is sometimes confused with white noise, as both cannot be predicted. But a random walk is different than white noise because it is not *entirely* random. Each value is altered from the value before it, just in a random direction. To create a random walk you could start with a random number, then randomly add or subtract 1 from it. Then, add or subtract 1 from that number, repeating the process until the time series is of the desired length.

In the real world, this resembles the day to day change in stock market prices, so we will use that for our random walk demonstration. Let's read in `MSFT.csv`, which contains stock prices for the Microsoft Stock over the past few years obtained from [Market Watch](https://www.marketwatch.com/investing/stock/msft/download-data?startDate=08/24/2019&endDate=08/24/2022), and examine some of the properties of the `Open` price. Let's look at the original time series first. 

In [ ]:
msft = pd.read_csv('MSFT.csv')
msft['Date'] = pd.to_datetime(msft['Date'])

plt.figure()
plt.plot(msft['Open'], c = 'k')
plt.title('MSFT Stock Price')

msft['Open'].describe()

The time series plot does not have the same random structure as that of the white noise. This series has distinct peaks and valleys, with distinct (but not necessarily cyclical or predictable) variations over time. The mean varies over time as well following these peaks and valleys. It is not zero, and it is not constant. 

However, we said that stock price **changes** represent a rolling walk, not the prices themselves. Let's use the `shift` function in pandas to shift all the `Open` data back 1 day, and then use that to compute `Open_diff` which is the day to day change in MSFT stock price. 

In [ ]:
msft['Open_diff'] = msft['Open'] - msft['Open'].shift(1)
msft.head()

In [ ]:
# get rolling average 
msft_roll = msft.set_index('Date').rolling(30).mean()
msft_roll.reset_index(inplace = True)
msft_roll['Date'] = pd.to_datetime(msft_roll['Date'])
msft_roll['offset'] = msft_roll['Date'] + to_offset('15D')

# plot 
plt.figure(figsize=(12, 4))
plt.plot(msft.Date, msft.Open_diff, c = 'k', label = 'Daily')
plt.grid(which = 'major')
plt.plot(msft_roll.offset, msft_roll.Open_diff, c = 'r', linewidth = 3, label = 'Rolling Average')
plt.title('MSFT Stock Price Changes')
plt.legend()

Now we are looking at the differences in stock price over time. 

However we have not proved that this is or is not a random walk series. To do so we once again need to look at an **autocorrelation** plot. 

<hr style="border:2px solid gray"> </hr>

### Now you try! 

Use the `.plot_acf()` function to produce an autocorrelation plot with a lag value of 50. Since we do have some missing values, set `missing = 'drop'` as an input argument so the function knows what to do when it encounters a `NaN`. 

In [ ]:
### BEGIN SOLUTION 

tsaplots.plot_acf(msft.Open_diff, lags=50, missing = 'drop');

### END SOLUTION 

<hr style="border:2px solid gray"> </hr>

Here we see that **we do have a random walk!** The differences in MSFT stock prices day to day are **not predictable**! Sometimes with a random walk you'll find some connection at the first lag, but that's not always the case. 

## Stationarity 

Another important quality to test for is **stationarity**, or if the time series is stationary or not. A stationary time series is **not affected by time** which means that the mean and standard deviation/variance remain constant over time. 

If a time series is **not stationary** it is because it either contains a trend or seasonality. Data with a **trend** means that the mean varies over time, usually in a particular direction. For example the population of a city would follow an upward trend as population grows. **Seasonality** means that there are variations following certain time-frames. For example, in a time series of temperatures in North America, you'd find consistent peaks in summers and dips in winters. 

By definition, all random walk data is stationary, including our MSFT data! But not all stationary data is a random walk. If you know already that your data is a random walk you do not need to test for stationarity as that is implied.  

Let's bring in some new data for us to work with to test for stationarity. We will read in `superstores.csv`, which contains sales data obtained [here](https://www.kaggle.com/datasets/rohitsahoo/sales-forecasting). We will do a bit of initial cleaning up, and isolate only the furniture data & select columns. There are multiple sales per day - we will also resample to create a dataframe with daily total sales. 

In [ ]:
superstores = pd.read_csv('superstores.csv')
superstores['Order Date'] = pd.to_datetime(superstores['Order Date'])
superstores = superstores[superstores['Category'] == 'Furniture']
superstores = superstores[['Order ID', 'Order Date', 'Customer ID', 'Category', 'Sales']]
superstores.sort_values('Order Date', inplace = True)

# resample 
daily_sales = superstores.set_index('Order Date').resample('D').sum()
daily_sales.reset_index(inplace = True)

# add cumulative sales 
daily_sales['Cumulative Sales'] = [sum(daily_sales['Sales'][0:i[0]]) for i in enumerate(daily_sales['Sales'].values)]

daily_sales.head()

The goal is to figure out of this is a stationary time series or not, in other words if it is time dependant. Let's start with some smoothing by implementing a 30 day rolling average. 

In [ ]:
# get rolling average
daily_sales_rolling = daily_sales.set_index('Order Date').rolling(60).mean()
daily_sales_rolling.reset_index(inplace = True)
daily_sales_rolling['Date Offset'] = daily_sales_rolling['Order Date'] - to_offset('30D')

# get rolling standard deviation 
daily_sales_stdv = daily_sales.set_index('Order Date').rolling(60).std()
daily_sales_stdv.reset_index(inplace = True)
daily_sales_stdv['Date Offset'] = daily_sales_stdv['Order Date'] - to_offset('30D')

# plot 
plt.figure(figsize=(12, 4))
plt.plot(daily_sales['Order Date'], daily_sales['Sales'], c = 'k', label = 'daily')
plt.plot(daily_sales_rolling['Date Offset'], daily_sales_rolling['Sales'], c = 'r', 
         linewidth = 4, label = '60 day rolling average')
plt.plot(daily_sales_stdv['Date Offset'], daily_sales_stdv['Sales'], c = 'y', 
         linewidth = 4, label = '60 day rolling standard deviation')
plt.title('Superstore Daily Sales')
plt.ylabel('Total Sales $')
plt.grid(which = 'major')
plt.legend()


While the mean and standard deviation aren't exactly constant through the entire span of the time series, they don't reveal anything that looks cyclical in nature. So maybe this is really a stationary time series! 

## The Dickey Fuller Test 

There is a statistical test we can use to confirm our eyeball conclusion, called the **Dickey Fuller Test**. This is available in `statsmodels` `stattools` package, in a function called `adfuller`.

The function takes the time series data as input (the sales data alone), and returns several things as listed in the [documentation](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html). Most important are the first value, which is the **test statistic**, and the fourth value which is a dictionary containing the **critical values**. The critical values are a 1%, 5%, and 10% confidence level. 

In short: 
- If the test statistic is **greater than** all the critical values, the time series is **stationary**. 
- If the test statitsic is **less than** the 1% critical values, there is a 99% confidence level that the time series is **not stationary**. 
- If the test statistic is greater than the 1% but less than the 5% critical values, there is a 95-99% chance the series is not stationary. 

Let's implement this test on our daily sales data and prove for sure if this is stationary or not. Since the list returned by `adfuller` is difficult to read, we will create a function that runs the test & includes an if statement that prints the result of the test in plain language. 

In [ ]:
def run_adfuller(timeseries):
    # drop nans
    timeseries = timeseries.dropna()
    # run test
    dftest  = stattools.adfuller(timeseries)
    # printer
    if dftest[0] < dftest[4]['1%']:
        print('the time series is stationary within a 99% confidence level')
    if dftest[0] > dftest[4]['1%']:
        print('the time series is not stationary')
    # return 
    return dftest

In [ ]:
run_adfuller(daily_sales.Sales)

## A non stationary time series 

Now what about a non-stationary time series? What might that look like? Let's take a look at the **cumulative** sales of the company over the same timeframe, which we created in the `Cumulative Sales` column in `daily_sales`. Let's recreate the time series plot for this data column showing the original data, rolling average, and rolling standard deviation. 

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(daily_sales['Order Date'], daily_sales['Cumulative Sales'], c = 'k', label = 'daily')
plt.plot(daily_sales_rolling['Date Offset'], daily_sales_rolling['Cumulative Sales'], c = 'r', 
         linewidth = 4, label = '60 day rolling average')
plt.plot(daily_sales_stdv['Date Offset'], daily_sales_stdv['Cumulative Sales'], c = 'y', 
         linewidth = 4, label = '60 day rolling standard deviation')
plt.title('Superstore Daily Sales')
plt.ylabel('Total Sales $')
plt.grid(which = 'major')
plt.legend()

It looks like while standard deviation is more or less constant, the rolling mean & the data itself both have a strong trend! As time goes on the company's sum of revenue continues to grow. A trend as strong as this means the time series surely cannot be stationary. 

<hr style="border:2px solid gray"> </hr>

### Now you try! 

Run the Dickey Fuller test on the `Cumulative Sales` column to confirm or deny that the time series is stationary. 

In [ ]:
### BEGIN SOLUTION 

# run test
run_adfuller(daily_sales['Cumulative Sales'])

### END SOLUTION     

<hr style="border:2px solid gray"> </hr>

## Differencing to remove a trend & make a series stationary

As we mentioned previously, there are several techniques that require a **stationary** time series for them to work. When dealing with a non-stationary series such as the `Cumulative Sales` above, we can utilize something called **differencing** to make it stationary! 

This includes using the `shift` method in pandas to take the difference between each value & the one that came previous. This is a **differencing transformation** that will make our series stationary by **removing the trend!** We confirm this by running the Dickey Fuller test once more on the transformed data. 

Looking back, this is exactly what we did to get the difference in MSFT stock prices! 

In [ ]:
# differencing 
daily_sales['Cumulative Sales diff'] = daily_sales['Cumulative Sales'] - daily_sales['Cumulative Sales'].shift(1)

# dickey fuller 
dftest = run_adfuller(timeseries = daily_sales['Cumulative Sales diff'])

The test confirms this is in fact stationary now! Let's take a look at the new shape and distribution below. The time series plot shows that the differencing has de-trended the data. In other words, it is no longer increasing over time. The average has become constant, which we know is true for stationary datasets. 

This new differenced dataset still contains the information content of the original (changes over time) but has been **decomposed** to remove the trend. We have transformed a non-stationary 

In [ ]:
fig, ax = plt.subplots(nrows = 2, figsize = (12, 7))
ax[0].plot(daily_sales['Order Date'], daily_sales['Cumulative Sales'], c = 'k')
ax[0].plot(daily_sales_rolling['Date Offset'], daily_sales_rolling['Cumulative Sales'], 
           linestyle = '--', c = 'r', linewidth = 3)
ax[0].set_title('Original Cumulative Sales')

ax[1].plot(daily_sales['Order Date'], daily_sales['Cumulative Sales diff'])
ax[1].plot(daily_sales_rolling['Date Offset'], daily_sales['Cumulative Sales diff'].rolling(60).mean(), 
          linestyle = '--', c = 'r', linewidth = 3)
ax[1].set_title('Differencing of Cumulative Sales')

fig.tight_layout()

## Reducing Noise Using Log & Square Root Transforms 

Exploratory time series analysis often revolves around trying to alter the series in order to reveal the underlying signal. We can **transform** our time series data a number of ways to help us reduce noise. Two of the simplest are using a **log transform** or a **square root** transform, where we simply take the log or square root of all values in the series to attempt to reduce noise. 

Depending on the data this may or may not work! As a general rule, log transforms work best for removing exponential growth in a time series (think 'hockey stick' plots), and square root transforms tend to work best to remove quadratic growth in a time series. Ultimately we want to remove the noise of the growth & make it more linear. 

Let's bring in a time series that shows growth to test theses transforms. Let's bring in a dataset which shows the number of airline passengers over time. Let's create a time series plot & histogram for it. 

In [ ]:
airline = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')
airline['Month'] = pd.to_datetime(airline['Month'])

airline_yr = airline.set_index('Month').resample('1Y').mean()
airline_yr.reset_index(inplace = True)
airline_yr['Date Offset'] = airline_yr['Month'] - to_offset('6M')

fig, ax = plt.subplots(ncols = 2, figsize = (12, 4))
ax[0].plot(airline['Month'], airline['Passengers'], label = 'monthly')
ax[0].plot(airline_yr['Date Offset'], airline_yr['Passengers'], label = 'yearly average')
ax[0].legend()
ax[1].hist(airline['Passengers'])

We see that this time series shows growth that doesn't look quite linear. We also see that this series also has **seasonality**, or a repeating cycle, which we identify based on the regular alternation of peaks and valleys. The histogram shows a left leaning skew. 

Let's apply the log and square root transformation & plot them to see the result. 

In [ ]:
# transform 
airline['Passengers log'] = np.log(airline['Passengers'])
airline['Passengers sqrt'] = np.sqrt(airline['Passengers'])

# plot 
plt.figure(figsize = (12, 4))
plt.plot(airline['Month'], airline['Passengers log'], label = 'log transform')
plt.plot(airline['Month'], airline['Passengers sqrt'], label = 'square root transform')
plt.legend()

fig, ax = plt.subplots(ncols = 2, figsize = (12, 4))
ax[0].hist(airline['Passengers sqrt'])
ax[0].set_title('square root transform')
ax[1].hist(airline['Passengers log'])
ax[1].set_title('log transform')

The square root transform seems to have reduced noise a bit but we still see the seasonal cycle and growth. This was not particularly helpful in demystifying this time series. The log transform however seems to have mostly removed the seasonal cycle and trend! It is nearly linear. Aside from reducing the noise in the time series, this indicates to us that the trend resembles exponential growth. 

The histograms also show that the square root transform still seems to have the left leaning skew, where the log transform looks more normally distributed. This is another sign that the noise has been reduced!


<hr style="border:2px solid gray"> </hr>

# Practice on your own 

For this practice we will be working with data on tractor sales, which we will read in from `tractor_sales.csv` which has a column indicating the `Month` and another indicating `Number Sold`.  

#### Exercise 1. Read in `tractor_sales.csv` and create an exploratory plot that includes a time series plot and a histogram. 

In [ ]:
### BEGIN SOLUTION 

tractor = pd.read_csv('tractor_sales.csv')
tractor['Date'] = pd.to_datetime(tractor['Date'])

fig, ax = plt.subplots(ncols = 2, figsize = (12, 4))
ax[0].plot(tractor['Date'], tractor['Number Sold'], label = 'monthly')
ax[0].legend()
ax[0].set_title('Tractor Sales')
ax[0].set_ylabel('Number Sold')
ax[1].hist(tractor['Number Sold'])
ax[1].set_title('Sales Distribution')

### END SOLUTION 

#### Exercise 2. Add a moving average & moving standard deviation to the time series plot. Given just an eyeball analysis of the plot, do you think this is a stationary or non-stationary time series?

In [ ]:
### BEGIN SOLUTION 

tractor_roll = tractor.set_index('Date').rolling(12).mean()
tractor_roll.reset_index(inplace = True)
tractor_roll['Date Offset'] = tractor_roll['Date'] - to_offset('6M')

tractor_stdv = tractor.set_index('Date').rolling(12).std()
tractor_stdv.reset_index(inplace = True)
tractor_stdv['Date Offset'] = tractor_stdv['Date'] - to_offset('6M')

plt.figure(figsize = (12, 4))
plt.plot(tractor['Date'], tractor['Number Sold'], label = 'monthly')
plt.plot(tractor_roll['Date Offset'], tractor_roll['Number Sold'], label = 'moving average')
plt.plot(tractor_stdv['Date Offset'], tractor_stdv['Number Sold'], label = 'standard deviation')
plt.legend()
plt.title('Tractor Sales')
plt.ylabel('Number Sold')

# this time series has a non-zero moving average that changes over time, it is probably not stationary. 

### END SOLUTION 

#### Exercise 3. Confirm or disprove your hypothesis using the Dickey Fuller test. Was your hypothesis correct? 

In [ ]:
### BEGIN SOLUTION 

dftest = run_adfuller(timeseries = tractor['Number Sold'])

# my hypothesis was correct, the dickey fuller test confirmed that it is not stationary 
# that means the test statistic was > the confidence interval

### END SOLUTION 

#### Exercise 4. Apply differencing to the time series to make it stationary. Plot this new time series along with the differenced moving average. What qualities of this plot imply that it is now stationary?

In [ ]:
### BEGIN SOLUTION 

tractor['Difference'] = tractor['Number Sold'] - tractor['Number Sold'].shift(1)

tractor_roll = tractor.set_index('Date').rolling(12).mean()
tractor_roll.reset_index(inplace = True)
tractor_roll['Date Offset'] = tractor_roll['Date'] - to_offset('6M')

plt.figure(figsize = (12, 4))
plt.plot(tractor['Date'], tractor['Difference'], label = 'monthly')
plt.plot(tractor_roll['Date Offset'], tractor_roll['Difference'], label = 'moving average')
plt.legend()
plt.title('Tractor Sales')
plt.ylabel('Number Sold')

### END SOLUTION 

#### Exercise 5. Create an corellogram for the differenced tractor sales values. Does this resemble a random walk? Why or why not?

In [ ]:
### BEGIN SOLUTION 

tsaplots.plot_acf(tractor['Difference'], lags=10, missing = 'drop');

# this does not resemble a random walk because the values are not all near-zero, there is 
# an outlier value of -0.75 at 6 months 

### END SOLUTION 

#### Exercise 6. Attempt to reduce the noise in our time series using either a log or square root transform. Which is more effective?

In [ ]:
### BEGIN SOLUTION 

# transform 
tractor['Sold log'] = np.log(tractor['Number Sold'])
tractor['Sold sqrt'] = np.sqrt(tractor['Number Sold'])

# plot 
plt.figure(figsize = (12, 4))
plt.plot(tractor['Date'], tractor['Sold log'], label = 'log transform')
plt.plot(tractor['Date'], tractor['Sold sqrt'], label = 'square root transform')
plt.legend()

fig, ax = plt.subplots(ncols = 2, figsize = (12, 4))
ax[0].hist(tractor['Sold sqrt'])
ax[0].set_title('square root transform')
ax[1].hist(tractor['Sold log'])
ax[1].set_title('log transform')

# the log transform is more effective because the transfomred time series most resembles a straight line, 
# and the histogram shows the values are more normally distributed

### END SOLUTION 